## Импорт библиотек

In [2]:
import datetime
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import time
from collections import deque

## Загрузка данных

In [3]:
df_raw= pd.read_csv(r"C:\Users\Stepan\Desktop\ВУЗ\Клиент-серверные\папка датафреймов\df_after_cleaning.csv")
df_raw=df_raw.drop('Unnamed: 0', axis=1)
df_raw.sample(5)

,metro_name,time_to_metro,num_rooms,total_area,year_house,kitchen_area,repair,price,floors,num_floor,address,date,coord_width,coord_len,link
16734,Мичуринский проспект,7.5,4.0,47.0,1700.0,5.5,косметический,29981046.0,46.0,34.0,"ул. Озёрная, вл. 1, стр. 4",2023-01-02,55.687600,37.471360,https://avito.ru//moskva/kvartiry/4-k._kvartir...
21833,Фили,5.0,1.0,20.0,2021.0,5.0,требует ремонта,17800000.0,14.0,11.0,"Москва, Большая Филёвская улица, 3к1",2023-01-05,55.751639,37.516251,https://avito.ru//moskva/kvartiry/1-k._kvartir...
24208,Верхние Лихоборы,2.5,1.0,21.0,1700.0,0.0,косметический,11668305.0,31.0,13.0,"Дмитровское ш., стр. 1",2023-01-09,55.858430,37.556230,https://avito.ru//moskva/kvartiry/1-k._kvartir...
15119,Улица Скобелевская,7.5,2.0,34.5,1700.0,4.5,евро,13400000.0,31.0,6.0,"Москва, улица Поляны, 5",2022-12-31,55.557765,37.555023,https://avito.ru//moskva/kvartiry/2-k._kvartir...
1555,Бабушкинская,25.5,1.0,22.5,2002.0,9.0,евро,11000000.0,14.0,3.0,"Москва, Полярная ул., 15к3",2023-04-24,55.875864,37.633751,https://www.avito.ru/moskva/kvartiry/1-k._kvar...


## Преобразование категориальных признаков.

Преобразование даты. Преобразуем в числовый признак, отражающий колиечство дней, прошедших со дня публикации.

In [4]:
def datedelta(x):
    y, m, d= [int(i) for i in x.split('-')]
    dat= datetime.date(y, m, d)
    return (datetime.date.today()-dat).days
df_raw['date']= df_raw['date'].transform(lambda x: datedelta(x))

Преобразование координат в расстояние до Кремля

In [5]:
def raddeg(r):
    d= r* 180 /np.pi
    return d

def degrad(d):
    r= d* np.pi / 180
    return r

def coord_distance(w1, l1, w2, l2):
    t= l1- l2
    d= 60* 1.1515* raddeg(np.arccos(
            (np.sin(degrad(w1))* np.sin(degrad(w2))) + 
            (np.cos(degrad(w1))* np.cos(degrad(w2)) * np.cos(degrad(t)))))    

    return round(d* 1.609344, 2)
df_raw['distance_to_Kremlin']=coord_distance(df_raw['coord_width'], df_raw['coord_len'], 55.751462, 37.618790)

Преобразование координат в район с помощью библиотеки geopy.

In [6]:
district=[]
w, l=list(df_raw['coord_width']), list(df_raw['coord_len'])
for i in range(len(w)):
    geolocator= Nominatim(user_agent='prohorovstepan17@gmail.com')
    coords= [w[i],l[i]] 
    location = geolocator.reverse(coords)
    district.append(location.raw['address'].get('suburb'))
    

Длина массива с районами

In [20]:
len(district)

27976

Размеры датасета

In [21]:
df_raw.shape

(27976, 16)

Присваиваем столбцу district массив district. И посмотрим на датасет

In [22]:
df_raw['district']=district
df_raw

,metro_name,time_to_metro,num_rooms,total_area,year_house,kitchen_area,repair,price,floors,num_floor,address,date,coord_width,coord_len,link,distance_to_Kremlin,district
0,Ясенево,8.0,0.0,26.0,1700.0,0.0,евро,3174900.0,16.0,16.0,"Москва, ул. Паустовского, 8к3",13,55.599967,37.536365,https://www.avito.ru/moskva/kvartiry/kvartira-...,17.62,район Ясенево
1,Улица академика Янгеля,31.0,0.0,10.0,1973.0,0.0,требует ремонта,2500000.0,12.0,1.0,"Москва, Медынская ул., 4/1",13,55.592490,37.645717,https://www.avito.ru/moskva/kvartiry/kvartira-...,17.76,район Бирюлёво Западное
2,Братиславская,25.5,0.0,14.5,1700.0,0.0,косметический,5700000.0,26.0,9.0,"ул. Люблинская, вл. 72, корп. 11",13,55.665770,37.734130,https://www.avito.ru/moskva/kvartiry/kvartira-...,11.96,район Люблино
3,Панфиловская,8.0,0.0,8.0,1960.0,0.0,дизайнерский,4900000.0,1.0,1.0,"Москва, ул. Маршала Конева, 2",13,55.799952,37.491889,https://www.avito.ru/moskva/kvartiry/kvartira-...,9.59,район Щукино
4,Рассказовка,31.0,0.0,20.0,2012.0,0.0,косметический,3290000.0,3.0,2.0,"Москва, Новомосковский административный округ,...",13,55.647214,37.327623,https://www.avito.ru/moskva/kvartiry/kvartira-...,21.62,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27971,Ростокино,5.0,5.0,75.0,2001.0,10.0,дизайнерский,42000000.0,17.0,17.0,"Москва, ул. Бажова, 24к2",118,55.836428,37.665533,https://avito.ru//moskva/kvartiry/5-k._kvartir...,9.89,Ростокино
27972,Ломоносовский проспект,2.5,4.0,70.5,2008.0,15.0,евро,74000000.0,14.0,11.0,"Москва, Ломоносовский пр-т, 29к1",118,55.703611,37.516476,https://avito.ru//moskva/kvartiry/4-k._kvartir...,8.33,район Раменки
27973,Тверская,5.0,5.0,115.5,1700.0,12.5,евро,99000000.0,4.0,4.0,"Москва, Большой Палашёвский пер., 2/12",118,55.763341,37.597073,https://avito.ru//moskva/kvartiry/5-k._kvartir...,1.89,Пресненский район
27974,Новаторская,5.0,6.0,129.5,1700.0,7.5,евро,72900000.0,29.0,6.0,"Москва, ул. Михаила Певцова",118,55.670855,37.514228,https://avito.ru//moskva/kvartiry/6-k._kvartir...,11.10,Проспект Вернадского


Количество Nan значений района

In [23]:
df_raw['district'].isna().sum()

2543

Заменим nan значения района на ''

In [24]:
df_raw['district']=df_raw['district'].fillna('')

Уберем слово район из названий районов

In [25]:
df_raw['district']=df_raw['district'].transform(lambda x: x.replace('район', '').strip())

Количество районов. В Москве их должно быть 126. У нас немного больше, наверняка мы захватили некоторые районы подмосковья. Но это не страшно.

In [26]:
len(df_raw.district.unique())

149

Сами районы по алфавиту

In [27]:
sorted(df_raw.district.unique())

['',
 'Академический',
 'Алексеевский',
 'Алтуфьевский',
 'Арбат',
 'Аэропорт',
 'Бабушкинский',
 'Басманный',
 'Беговой',
 'Бескудниковский',
 'Бибирево',
 'Бирюлёво Восточное',
 'Бирюлёво Западное',
 'Богородское',
 'Братеево',
 'Бусиново',
 'Бутово',
 'Бутырский',
 'Верхние Лихоборы',
 'Вешняки',
 'Войковский',
 'Восточное Дегунино',
 'Восточное Измайлово',
 'Выхино-Жулебино',
 'Гавриково',
 'Гагаринский',
 'Головинский',
 'Гольяново',
 'Даниловский',
 'Дегунино',
 'Дмитровский',
 'Донской',
 'Дорогомилово',
 'Дрожжино',
 'Елохово',
 'Замоскворечье',
 'Западное Дегунино',
 'Зенино',
 'Зюзино',
 'Зябликово',
 'Ивановское',
 'Измайлово',
 'Канатчиково',
 'Капотня',
 'Кожухово',
 'Коломенка',
 'Коньково',
 'Коптево',
 'Косино',
 'Косино-Ухтомский',
 'Котловка',
 'Красная горка',
 'Красносельский',
 'Крылатское',
 'Крюково',
 'Кузьминки',
 'Кунцево',
 'Куркино',
 'Курьяново',
 'Левобережный',
 'Лефортово',
 'Лианозово',
 'Ломоносовский',
 'Лосиноостровский',
 'Люблино',
 'Марфино',
 'Ма

Преобразование названия метро в ветку и расстояния до центральных станций

In [28]:
metro_dict={"Арбатско-Покровская": ['Пятницкое шоссе', 'Митино', 'Волоколамская', 'Мякинино','Строгино', 'Крылатское', 'Молодёжная',
'Кунцевская', 'Славянский бульвар',
'Парк Победы', 'Киевская', 'Смоленская', 'Арбатская',
'Площадь революции','Курская','Бауманская',
'Электрозаводская','Семёновская','Партизанская','Измайловская','Первомайская', 'Щёлковская'],
"Бутовская":

[
    'Улица Горчакова',
    'Бульвар адмирала Ушакова',
 'Улица Скобелевская',
 'Улица Старокачаловская',
 'Лесопарковая',
 'Битцевский парк',




],
'Замоскворецкая':

['Ховрино','Беломорская','Речной вокзал','Водный стадион',
 'Войковская','Сокол', 'Аэропорт', 'Динамо', 'Белорусская', 'Маяковская',
 'Тверская', 'Театральная', 'Новокузнецкая', 'Павелецкая', 'Автозаводская',
 'Технопарк', 'Коломенская', 'Каширская', 'Кантемировская', 'Царицыно', 'Орехово',
 'Домодедовская','Красногвардейская','Алма-Атинская'
],
 
'Калининско-Солнцевская':[
    'Новокосино',
    'Новогиреево',
    'Перово',
    'Шоссе Энтузиастов',
    'Авиамоторная',
    'Площадь Ильича',
    'Марксистская',
    'Третьяковская'
],
'Калужско-Рижская':
[
'Новоясеневская',
'Ясенево',
'Тёплый стан',
'Коньково',
'Беляево',
'Калужская',
'Новые Черёмушки',
'Профсоюзная',
'Академическая',
'Ленинский проспект',
'Шаболовская',
'Октябрьская',
'Третьяковская',
'Китай-город',
'Тургеневская',
'Сухаревская',
'Проспект Мира',
'Рижская',
'Алексеевская',
'ВДНХ',
'Ботанический сад',
'Свиблово',
'Бабушкинская',
'Медведково'],
 
'Люблинско-Дмитровская':
[
'Зябликово',
'Шипиловская',
'Борисово',
'Марьино',
'Братиславская',
'Люблино',
'Волжская',
'Печатники',
'Кожуховская',
'Дубровка',
'Крестьянская застава',
'Римская',
'Чкаловская',
'Сретенский бульвар',
'Трубная',
'Достоевская',
'Марьина роща',
'Бутырская', 
'Фонвизинская', 
'Петровско-Разумовская', 
'Окружная', 
'Верхние Лихоборы',
'Селигерская'],
 
'Серпуховско-Тимирязевская':
[
'Бульвар Дмитрия Донского',
'Аннино',
'Улица академика Янгеля',
'Пражская',
'Южная',
'Чертановская',
'Севастопольская',
'Нахимовский Проспект',
'Нагорная',
'Нагатинская',
'Тульская',
'Серпуховская',
'Полянка',
'Боровицкая',
'Чеховская',
'Цветной бульвар',
'Менделеевская',
'Савеловская',
'Дмитровская',
'Тимирязевская',
'Владыкино',
'Отрадное',
'Бибирево',
'Алтуфьево'
],
 
'Солнцевская':
[
'Рассказовка',
'Новопеределкино',
'Боровское шоссе',
'Солнцево',
'Говорово',
'Озёрная',
'Мичуринский проспект',
'Раменки',
'Ломоносовский проспект',
'Минская',
'Парк Победы',
'Деловой центр'],
'Сокольническая':

['Бульвар Рокоссовского',
'Черкизовская',
'Преображенская площадь',
'Сокольники',
'Красносельская',
'Комсомольская',
'Красные ворота',
'Чистые пруды',
'Лубянка',
'Охотный ряд',
'Библиотека им. Ленина',
'Кропоткинская',
'Парк Культуры',
'Фрунзенская',
'Спортивная',
'Воробьёвы горы',
'Университет',
'Проспект Вернадского',
'Юго-Западная',
'Тропарёво',
'Румянцево',
'Саларьево'],
 
'Таганско-Краснопресненская':
[
'Котельники',
'Жулебино',
'Лермонтовский проспект',
'Выхино',
'Рязанский проспект',
'Кузьминки',
'Текстильщики',
'Волгоградский проспект',
'Пролетарская',
'Таганская',
'Китай-город',
'Кузнецкий мост',
'Пушкинская',
'Баррикадная',
'Улица 1905 года',
'Беговая',
'Полежаевская',
'Октябрьское поле',
'Щукинская',
'Спартак',
'Тушинская',
'Сходненская',
'Планерная'],
'Филевская':
[
'Александровский сад',
'Арбатская',
'Смоленская',
'Киевская',
'Студенческая',
'Кутузовская',
'Фили',
'Багратионовская',
'Филевский парк',
'Пионерская',
'Кунцевская',
'Выставочная',
'Международная'],
 
'Кольцевая':
['Парк культуры',
'Октябрьская',
'Калужская ',
'Добрынинская',
'Павелецкая',
'Таганская',
'Курская',
'Комсомольская',
'Проспект Мира',
'Новослободская',
'Краснопресненская',
'Киевская'],
'МЦК':
[
    'Хорошево',
'Новопесчаная',
'Стрешнево',
'Лихоборы',
'Войковская',
'Балтийская',
'Николаевская',
'Окружная',
'Владыкино',
'Ботанический сад',
'Ростокино',
'Белокаменная',
'Бульвар Рокоссовского',
'Локомотив',
'Измайлово',
'Соколиная гора',
'Шоссе энтузиастов',
'Андроновка',
'Угрешская',
'Новохохловская',
'Нижегородская',
'Дубровка',
'Автозаводская',
'ЗИЛ',
'Верхние котлы',
'Крымская',
'Площадь Гагарина',
'Лужники',
'Кутузовская',
'Деловой центр',
'Шелепиха'
],
'Некрасовская':
[
'Электрозаводская',
'Лефортово',
'Авиамоторная',
'Нижегородская',
'Стахановская',
'Окская',
'Юго-Восточная',
'Косино',
'Улица Дмитриевского',
'Некрасовка',
'Лухмановская'
]
      }
dict_metro={i:k for k, v in metro_dict.items() for i in v}
           
           


central_stations={"Арбатско-Покровская":11, "Бутовская":19, "Калужско-Рижская":13,'Замоскворецкая': 10, 
                  'Калининско-Солнцевская': 7, 'Люблинско-Дмитровская':11, 'Серпуховско-Тимирязевская':11,
                  'Солнцевская':11, 'Сокольническая':8, 'Таганско-Краснопресненская':11,'Филевская':0, 'Некрасовская':1
                 }
def metro_line(x):
    try:
        return dict_metro[x.strip()]
    except:
        return 'БКЛ'
def metro_distance(x):
    
    try:
        k=dict_metro[x]
    except:
        k='БКЛ'
    if k=='БКЛ':
        return 6
    if k=='МЦК':
        return 4
    if k=='Кольцевая':
        return 2
    return abs(central_stations[k]-metro_dict[k].index(x))
    
df_raw['metro_line']=list(map(lambda x: metro_line(x), df_raw['metro_name']))
df_raw['metro_distance']=list(map(lambda x: metro_distance(x), df_raw['metro_name']))

Итоговый датафрейм

In [31]:
df_raw

,metro_name,time_to_metro,num_rooms,total_area,year_house,kitchen_area,repair,price,floors,num_floor,address,date,coord_width,coord_len,link,distance_to_Kremlin,district,metro_line,metro_distance
0,Ясенево,8.0,0.0,26.0,1700.0,0.0,евро,3174900.0,16.0,16.0,"Москва, ул. Паустовского, 8к3",13,55.599967,37.536365,https://www.avito.ru/moskva/kvartiry/kvartira-...,17.62,Ясенево,Калужско-Рижская,12
1,Улица академика Янгеля,31.0,0.0,10.0,1973.0,0.0,требует ремонта,2500000.0,12.0,1.0,"Москва, Медынская ул., 4/1",13,55.592490,37.645717,https://www.avito.ru/moskva/kvartiry/kvartira-...,17.76,Бирюлёво Западное,Серпуховско-Тимирязевская,9
2,Братиславская,25.5,0.0,14.5,1700.0,0.0,косметический,5700000.0,26.0,9.0,"ул. Люблинская, вл. 72, корп. 11",13,55.665770,37.734130,https://www.avito.ru/moskva/kvartiry/kvartira-...,11.96,Люблино,Люблинско-Дмитровская,7
3,Панфиловская,8.0,0.0,8.0,1960.0,0.0,дизайнерский,4900000.0,1.0,1.0,"Москва, ул. Маршала Конева, 2",13,55.799952,37.491889,https://www.avito.ru/moskva/kvartiry/kvartira-...,9.59,Щукино,БКЛ,6
4,Рассказовка,31.0,0.0,20.0,2012.0,0.0,косметический,3290000.0,3.0,2.0,"Москва, Новомосковский административный округ,...",13,55.647214,37.327623,https://www.avito.ru/moskva/kvartiry/kvartira-...,21.62,,Солнцевская,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27971,Ростокино,5.0,5.0,75.0,2001.0,10.0,дизайнерский,42000000.0,17.0,17.0,"Москва, ул. Бажова, 24к2",118,55.836428,37.665533,https://avito.ru//moskva/kvartiry/5-k._kvartir...,9.89,Ростокино,МЦК,4
27972,Ломоносовский проспект,2.5,4.0,70.5,2008.0,15.0,евро,74000000.0,14.0,11.0,"Москва, Ломоносовский пр-т, 29к1",118,55.703611,37.516476,https://avito.ru//moskva/kvartiry/4-k._kvartir...,8.33,Раменки,Солнцевская,3
27973,Тверская,5.0,5.0,115.5,1700.0,12.5,евро,99000000.0,4.0,4.0,"Москва, Большой Палашёвский пер., 2/12",118,55.763341,37.597073,https://avito.ru//moskva/kvartiry/5-k._kvartir...,1.89,Пресненский,Замоскворецкая,0
27974,Новаторская,5.0,6.0,129.5,1700.0,7.5,евро,72900000.0,29.0,6.0,"Москва, ул. Михаила Певцова",118,55.670855,37.514228,https://avito.ru//moskva/kvartiry/6-k._kvartir...,11.10,Проспект Вернадского,БКЛ,6


## Сохранение данных

In [30]:
df_raw.to_csv(r"C:\Users\Stepan\Desktop\ВУЗ\Клиент-серверные\папка датафреймов\df_after_work_with_cat1.csv")